In [1]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as img
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
#from keras.callbacks import ModelCheckpoint
#print(tf.VERSION)
#from keras.callbacks import CSVLogger

Using TensorFlow backend.


In [ ]:
#Due to the model was trained on differen resourses such as Colab etc. It was necessary to zip/unzip data
import os
import zipfile

local_zip = 'train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('train')
local_zip = 'test.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('test')
zip_ref.close()

In [ ]:
K.clear_session()
sess = tf.Session(config=config)
K.set_session(sess)
nb_train_samples = 750 
nb_validation_samples = 250 
batch_size = 32
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=30, # 6-th from 25 to 30
    zoom_range=0.5, # 7-th from 0.3 to 0.5
    fill_mode='constant',
    rotation_range=180, #6-rd itteration from 90 to 180
    width_shift_range=.25, # 5-th from 0.5 to 0.25
    height_shift_range=.25, # 5-th from 0.5 to 0.25
    horizontal_flip=True)
    #vertical_flip=True,
    #channel_shift_range=100., 3-rd itteration
    #brightness_range=[0.5,1.0], 6-th 
    #zca_whitening=True) # 6-th

validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        'train/train/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x150
        batch_size = batch_size,
        # Since we use 32 classes, we need class_mode='categorical'
        class_mode='categorical')

# Flow training images in batches of 32 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        'test/test/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size = batch_size,
        # Since we use 32 classes, we need class_mode='categorical'
        class_mode='categorical')
inception = InceptionV3(weights='imagenet', include_top=False)
# Frozing the model layers. For our case it's not working
#for layer in inception.layers:
#  layer.trainable = False
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x) # 7-th 

predictions = Dense(32,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)
model = Model(inputs=inception.input, outputs=predictions)
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.001, momentum=0.9), metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='best_model_44class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_44class.log')

Found 24000 images belonging to 32 classes.
Found 8000 images belonging to 32 classes.


In [ ]:
model.summary()

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch = nb_train_samples // batch_size,
      validation_data = validation_generator,
      validation_steps = nb_validation_samples // batch_size,  
      epochs=150,
      verbose=1
     #callbacks=[csv_logger, checkpointer]
)

In [ ]:
def plot_accuracy(history,title):
    plt.title(title)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()
def plot_loss(history,title):
    plt.title(title)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()
plot_accuracy(history,'FOOD101_32 classes')
plot_loss(history,'FOOD101_32 classes')